In [ ]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [ ]:
#
#study_id = GALAXY_INPUTS['study_id']


#
galaxyInput_rankToMerge = GALAXY_INPUTS["RankToMergeOn"]
# selection from "superkingdom" "kingdom" "phylum" "class" "order" "family" "genus" "species" "all" "only counts"

galaxyInput_countTables = GALAXY_INPUTS["CountTables"]


mappingFromTaxaToNumber = {
    "superkingdom":1,
    "kingdom":2,
    "phylum":3,
    "class":4,
    "order":5,
    "family":6,
    "genus":7,
    "species":8,
    "all" : 10,         # to prevent key error in converting rank to number
    "only counts" : 10  # only counts and all will be handled when creating the tables
}

rankToMerge = mappingFromTaxaToNumber[galaxyInput_rankToMerge]



filePaths = []

for inputTable in galaxyInput_countTables:
    filePaths.append(inputTable["path"])


In [ ]:
tables = []
for file in filePaths:
    # Read out all Tables and remove unneccecary informations
    df = pd.read_csv(file,sep="\t",header=1)
    df = df.iloc[: , 1:]
    if("taxid" in df.columns):
        df = df.drop(columns=["taxid"])
    df.columns = [file.split("/")[-1].replace(".tsv",""),"#KEY"]
    columns = df.columns.tolist()
    columns.reverse()
    df = df[columns]
    tables.append(df)
# Merge all tables into one
df = reduce(lambda df1,df2:pd.merge(df1,df2,on="#KEY",how="outer"),tables)
df = df.fillna(0)
df

In [ ]:
if(galaxyInput_rankToMerge == "only counts"):
    df.to_csv("outputs/collection/countTable.tsv",sep="\t",index=False)

In [ ]:
if(rankToMerge < 10):
    df2 = df
    df2 = df2.reset_index()
    for index, row in df.iterrows():
        rowRaw = row['#KEY']
        rowSplit = rowRaw.split(";")
        if(len(rowSplit) >= rankToMerge):
            df2.loc[index,"#KEY"] = rowSplit[rankToMerge-1].replace("__","_taxa_")
        else:
            df2.loc[index,"#KEY"] = "noValue"

    df2 = df2[df2["#KEY"] != "noValue"]
    df2 = df2.groupby(["#KEY"]).sum()
    df2 = df2.reset_index()
    df2 = df2.drop(columns=["index"])
    df2.to_csv(f"outputs/collection/mergedOn{galaxyInput_rankToMerge}.tsv",sep="\t",index=False)

In [ ]:
if(galaxyInput_rankToMerge == "all"):
    tables = []
    for i in range(2,9):
        df2 = df
        df2 = df2.reset_index()
        for index, row in df.iterrows():
            rowRaw = row['#KEY']
            rowSplit = rowRaw.split(";")
            if(len(rowSplit) >= i):
                df2.loc[index,"#KEY"] = rowSplit[i-1].replace("__","_taxa_")
            else:
                df2.loc[index,"#KEY"] = "noValue"

        df2 = df2[df2["#KEY"] != "noValue"]
        df2 = df2.groupby(["#KEY"]).sum()
        df2 = df2.reset_index()
        df2 = df2.drop(columns=["index"])
        tables.append(df2)
    result = pd.concat(tables)
    result.to_csv(f"outputs/collection/mergedOn{galaxyInput_rankToMerge}.tsv",sep="\t",index=False)